In [68]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [69]:
# imports
import sys
sys.path.append("..")
import os
import shutil

import torch
from torch.distributions.multivariate_normal import MultivariateNormal

import matplotlib.pyplot as plt

In [70]:
# set these parameters to create all files
n_data = int(1e4)
folder = f'../data/{n_data:.1e}/'
import utils.model3
model = utils.model3.Net_mask()
n_bnn = 100

In [71]:
# run this to clean folder 
path_to_output_folder = folder
if os.path.exists(path_to_output_folder) and os.path.isdir(path_to_output_folder):
    shutil.rmtree(path_to_output_folder)
os.makedirs(path_to_output_folder)

In [72]:
path_to_output_folder = f'{folder}data/' 
if os.path.exists(path_to_output_folder) and os.path.isdir(path_to_output_folder):
    shutil.rmtree(path_to_output_folder)
os.makedirs(path_to_output_folder)

n_param = 10
data_cov = 1
data_mean = torch.zeros(n_param)
data_covariance = torch.eye(n_param)*data_cov

torch.manual_seed(1)
tensor_data = torch.zeros(n_data, 2*n_param)
tensor_d_sample =  MultivariateNormal(loc=data_mean, covariance_matrix=data_covariance).sample(sample_shape=(n_data,))
tensor_n_masked = torch.randint(n_param, (n_data,))
tensor_masks = torch.rand(n_data, n_param).argsort(dim=1)
tensor_masks = (tensor_masks < tensor_n_masked.unsqueeze(1))*1
tensor_data[:,:n_param] = tensor_d_sample*tensor_masks
tensor_data[:,n_param:] = tensor_masks
torch.save(tensor_data, f'{path_to_output_folder}data.pt')

In [73]:
tensor_D = torch.tensor(
    [[1, -1, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 1, -1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0,  1, -1, 0, 0, 0, 0, 0, 0],
    [0, 0,  0, 1, -1, 0, 0, 0, 0, 0],
    [0, 0,  0, 0, 1, -1, 0, 0, 0, 0],
    [0, 0,  0, 0, 0, 1, -1, 0, 0, 0],
    [0, 0,  0, 0, 0, 0, 1, -1, 0, 0],
    [0, 0,  0, 0, 0, 0, 0, 1, -1, 0],
    [0, 0,  0, 0, 0, 0, 0, 0, 1, -1],
    [-1, 0, 0, 0, 0, 0, 0, 0, 0, 1],], dtype=torch.float
)
tensor_Q_m = torch.mm(torch.t(tensor_D), tensor_D) 
n_param = tensor_D.size(dim=0)
vu = 1000
inversion_scaling = 1/vu
tensor_Q_m_modified = tensor_Q_m + torch.eye(n_param)*inversion_scaling 
tensor_Sigma_m = torch.inverse(tensor_Q_m_modified)

In [74]:
path_to_output_folder = f'{folder}rml_noise/' 
if os.path.exists(path_to_output_folder) and os.path.isdir(path_to_output_folder):
    shutil.rmtree(path_to_output_folder)
os.makedirs(path_to_output_folder)

sigma2 = 0.1**2
tau2 = 0.3**2
n_weights = sum(p.numel() for p in model.parameters())

for i in range(n_bnn):
    print(i)
    path = f'{path_to_output_folder}{i}/'
    os.makedirs(path)
    noise_data = MultivariateNormal(loc=torch.zeros(n_param), covariance_matrix=(torch.eye(n_param)*sigma2)).sample(sample_shape=(n_data,))
    noise_data_regularization =  MultivariateNormal(loc=torch.zeros(n_param), covariance_matrix=(tensor_Sigma_m)).sample(sample_shape=(n_data,))
    noise_parameter = torch.t(MultivariateNormal(loc=torch.zeros(1), covariance_matrix=(torch.eye(1)*tau2)).sample(sample_shape=(n_weights,)))
    torch.save(noise_data, f'{path}noise_data.pt')
    torch.save(noise_data_regularization, f'{path}noise_data_regularization.pt')
    torch.save(noise_parameter, f'{path}noise_parameter.pt')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
